# Shots vs Success Dashboard

Shor アルゴリズム(量子版)のショット数と成功率/ゲート指標を素早く評価するためのノートブックです。
1. 下セルの設定を調整して DataFrame を生成
2. 成功率を可視化してノイズ/mitigation 効果を比較
3. 必要に応じて DataFrame を CSV に出力して別分析へ渡してください


In [ ]:
import sys
from pathlib import Path
repo_root = Path().resolve()
if (repo_root / 'src').exists():
    project_root = repo_root
else:
    project_root = repo_root.parent
sys.path.insert(0, str(project_root))


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.noise.errors import ReadoutError

from src.quantum_rsa.experiment_logging import QuantumRunSetting, summarize_success, sweep_shot_counts
from src.quantum_rsa.shor_demo import DEFAULT_N_COUNT


In [ ]:
def build_simple_noise_model(
    single_qubit_prob: float = 0.002,
    two_qubit_prob: float = 0.02,
    readout_prob: float = 0.03,
) -> NoiseModel:
    """Depolarizing + readout noise helper for AerSimulator."""
    noise = NoiseModel()
    single_error = depolarizing_error(single_qubit_prob, 1)
    two_qubit_error = depolarizing_error(two_qubit_prob, 2)
    single_targets = ['h', 'x', 'y', 'z', 's', 'sdg', 't', 'tdg', 'rx', 'ry', 'rz', 'u', 'u1', 'u2', 'u3']
    two_targets = ['cx', 'cz', 'swap']
    noise.add_all_qubit_quantum_error(single_error, single_targets)
    noise.add_all_qubit_quantum_error(two_qubit_error, two_targets)
    meas_error = ReadoutError([[1 - readout_prob, readout_prob], [readout_prob, 1 - readout_prob]])
    noise.add_all_qubit_readout_error(meas_error)
    noise.name = "depolarizing+readout"
    return noise

noise_model = build_simple_noise_model()
noise_model


In [ ]:
shots = [256, 512, 1024, 2048, 4096]
settings = [
    QuantumRunSetting(label="ideal"),
    QuantumRunSetting(label="noisy", noise_model=noise_model),
    QuantumRunSetting(label="noisy+mitigated", noise_model=noise_model, apply_readout_mitigation=True),
]

df = sweep_shot_counts(number=15, base=7, shots_list=shots, repeats=3, settings=settings)
df.head()


In [ ]:
summary = summarize_success(df)
fig, ax = plt.subplots(figsize=(6, 4))
for label, chunk in summary.groupby('label'):
    ax.plot(chunk['shots'], chunk['success_rate'], marker='o', label=label)

ax.set_xlabel('Shots')
ax.set_ylabel('Success rate')
ax.set_ylim(0, 1.05)
ax.set_xscale('log', base=2)
ax.legend()
ax.grid(True, which='both', ls='--', alpha=0.4)
summary
